In [1]:
from resevit_road import restevit_road_cls,restevit_road_cls_lightweight
from torchsummary import summary
import torch
from fvcore.nn import FlopCountAnalysis,flop_count_table
from Load_data import Torch_load
from torch import nn
from tqdm import tqdm
from torch import optim
from Report_model.Report_Torch import Class_Report

IndentationError: unindent does not match any outer indentation level (Torch_load.py, line 140)

In [ ]:
model=restevit_road_cls_lightweight(num_class=4)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model= model.to(device)
summary(model, (3,224,224), batch_size=1)
inputs= torch.rand(1,3, 224,224)
inputs=inputs.to(device)
flops=FlopCountAnalysis(model,inputs)
flops.total()
flop_count_table(flops)

In [3]:
def save_checkpoint(model, filename):
    torch.save(model.state_dict(), filename)
    
output_log="output.txt"
def train(epochs_per_fold=30,batch_size=32,image_size=224):
    for fold,(dataloader_dict) in enumerate(Torch_load(image_size=image_size,batch_size=batch_size,task="multi")()):
        print("Fold:",fold+1)
        criterior= nn.CrossEntropyLoss()
        if fold<1: optimizer = optim.Adam(model.parameters(), lr=0.0001)
        else: optimizer = optim.Adam(model.parameters(), lr=0.00001)
        early_stop_thresh = 5
        best_accuracy = -1
        best_epoch = -1
        stop=False
        for epoch in range(epochs_per_fold):
            if stop==True: break
            with open(output_log, 'a') as f:
                print("Epoch {}/{}".format(epoch, epochs_per_fold),file=f)
            for phase in ["Train", "Val"]:
                if phase == "Train":
                    model.train()
                else:
                    model.eval()
                    
                epoch_loss = 0.0
                epoch_corrects = 0
                
                for inputs, labels in tqdm(dataloader_dict[phase]):
                    inputs = inputs.to(device)
                    labels = labels.to(device)
                    optimizer.zero_grad()
                    
                    with torch.set_grad_enabled(phase == "Train"):
                        outputs = model(inputs)
                        loss = criterior(outputs, labels)
                        _, preds = torch.max(outputs, 1)
                        
                        if phase == "Train":
                            loss.backward()
                            optimizer.step()
                            
                    epoch_loss += loss.item()*inputs.size(0)
                    epoch_corrects += torch.sum(preds==labels.data)
                epoch_loss = epoch_loss / len(dataloader_dict[phase].dataset)
                epoch_accuracy = epoch_corrects.double() / len(dataloader_dict[phase].dataset)
                if phase == "Val":
                    if epoch_accuracy > best_accuracy:
                        best_accuracy = epoch_accuracy
                        best_epoch = epoch
                        save_checkpoint(model, "ResEViT_multiclass.pth")
                    elif epoch - best_epoch > early_stop_thresh:
                        print("Early stopped training at epoch %d" % epoch)
                        stop=True  # terminate the training loo
                with open(output_log, 'a') as f:
                    print("{} Loss: {:.4f} Acc: {:.4f}".format(phase, epoch_loss, epoch_accuracy), file=f)

In [4]:
train()

Fold: 1


100%|██████████| 13/13 [00:03<00:00,  3.77it/s]


Early stopped training at epoch 8
Fold: 2


100%|██████████| 13/13 [00:03<00:00,  3.53it/s]


Early stopped training at epoch 12
Fold: 3


...

Early stopped training at epoch 6
Fold: 4


100%|██████████| 13/13 [00:04<00:00,  3.24it/s]


Early stopped training at epoch 6
Fold: 5


100%|██████████| 13/13 [00:03<00:00,  4.18it/s]

Early stopped training at epoch 7


In [2]:
checkpoint = torch.load('ResEViT_multiclass.pth')
model.load_state_dict(checkpoint)
Test_data=Torch_load(image_size=224,batch_size=8,phase="Test",task="multi")()
Class_Report(Test_data["Test"],model,device)()

NameError: name 'model' is not defined